In [ ]:
#!pip install python-dotenv

# TODO
- Odd one out with numbers of answers
- [urgent] Word topic matching is very wrong format -> do numbers make sense here?
- There should be free text answers!
- [urgent] for gap text, implement free text answers more clearly. sometimes the gap number repeats
  - test the free text answer formatting here
  - sometimes provides correct answers in the correct order
- code the level upgrade in python logic
- code the level assessment somehow -> there should be a way to input the language level

In [1]:
import os

import dotenv
dotenv.load_dotenv()

import sys
sys.path.append('..')
import temimbo

import importlib

importlib.reload(temimbo)

<module 'temimbo' from 'c:\\Users\\franz\\OneDrive\\Desktop\\BA\\BA\\LanguageTeacher\\notebooks\\..\\temimbo\\__init__.py'>

In [2]:
connector_openai = temimbo.ConnectorOpenAI(
    openai_key = os.getenv('OPENAI_KEY')
)
tg = temimbo.TaskGenerator(connector_llm = connector_openai)
evaluator = temimbo.AnswerEvaluator(connector_llm = connector_openai)


ui = temimbo.UserInterface()
db = temimbo.DatabaseClientLocalFile(connection_string='./database')
formater = temimbo.Formater()



In [3]:
profile = await db.load_profile(id='Franzy')
domain = await ui.choose_domain(domain='text')
task_type = await ui.choose_task_type(task_type='text_summary')
level, training_goals_subset = await tg.incorporate_profile_in_task(profile, domain)

In [4]:
prompt = await tg.generate_prompt(
    level = level,
    training_goals_subset = training_goals_subset,
    domain = domain,
    task_type = task_type,
)
print(prompt)

Pretend you are a academic english teacher. Generate a task description, and the task according to the format of the examples. No other text or tasks.
Generate a short text that the reader has to summarize in 5 to 6 sentences. only give a text of at least 250 words and a task description asking the student to write a summary, do not write a summary yourself.. Generate it for the area of text teaching, targeting on scientific developments.
Adjust the task dificulty to language level C2 (Can understand with ease virtually everything heard or read. Can summarise information from different spoken and written sources, reconstructing arguments and accounts in a coherent presentation. Can express him/herself spontaneously, very fluently and precisely, differentiating finer shades of meaning even in more complex situations).


In [5]:
raw_output = await tg.generate_task(prompt)
print(raw_output)

Text:

Scientists have recently made a ground-breaking discovery in the field of quantum physics. Using advanced computer simulations, they have successfully identified a new state of matter known as time crystals. Time crystals are unique because they exhibit a phenomenon known as time-translation symmetry breaking, which means that they oscillate between two different states without using any energy. This contradicts the laws of thermodynamics, which state that in order for something to oscillate, it must use energy. The discovery of time crystals has important implications for the field of quantum computing, as they could be used to create more stable qubits, which are the basic units of information in a quantum computer. This could lead to significant advancements in computing power and speed. Additionally, time crystals may also play a role in the development of new materials and technologies, as their unique properties could be harnessed for a variety of applications. Overall, th

In [6]:
formatted_output_task = await formater.output_task_formatting(raw_output)
user_answer = await ui.answer_task(formatted_output_task, answer="Storage logistics is a critical factor in the success of any business. It involves managing inventory, warehousing, transportation and distribution to ensure that products are delivered on time and within budget. Improper storage logistics can lead to delays in product delivery, increases in operational costs, and decreases in customer satisfaction. The complexity of modern supply chain systems makes it essential for companies to have an effective storage logistics management system in place to achieve their desired goals and objectives.")
formated_user_answer = await formater.learner_answer_formatting(user_answer)


In [7]:
######################
# This is what it will actually looks like
NL_feedback, correctness, training_goals = await evaluator.evaluate_learner_answer(
    domain = domain,
    formatted_output_task = formatted_output_task,
    formated_user_answer = formated_user_answer,
    task_type = task_type,
)
print('NL_feedback =', NL_feedback)
print('correctness =', correctness)
print('training_goals =', training_goals)

NL_feedback = Thank you for your response. However, your answer is not correct for the given task. You have provided information about storage logistics, which is not relevant to the text about time crystals and quantum physics. 

To improve, I would suggest focusing on the key points of the text, including the description of time crystals, their unique properties, and their potential implications for quantum computing, materials development, and technological applications. Additionally, practice using the vocabulary related to quantum physics, such as time-translation symmetry breaking, thermodynamics, qubits, computing power, and technological advancements.

The correct answer for the summary of the text is:
Scientists have made a ground-breaking discovery in quantum physics - they have identified a new state of matter called time crystals. Time crystals oscillate between two different states without using any energy, which contradicts the laws of thermodynamics. These time crystals 

In [9]:
# While the parsing is not implemented, this function will not work
new_profile = await evaluator.update_learner_profile(training_goals, profile)
#print(f'Old profile:\n{profile}\n\nNew profile:\n{new_profile}')

In [10]:
# This overwrites the original profile
# For testing purposes, it's easier to just comment it out
await db.save_profile(new_profile)